In [2]:
from openskill.models import PlackettLuce
import sqlite3 as sq
import pandas as pd

In [3]:
con = sq.connect('stats2.db')
qbr = sq.connect('qbr.db')
pg = pd.read_sql("SELECT * FROM player_games", con)
tg = pd.read_sql("""
SELECT sets.year, tournaments.tournament_name, tournaments.date, team_games.* FROM team_games 
LEFT JOIN tournaments on team_games.tournament_id = tournaments.tournament_id
LEFT JOIN sets on team_games.set_id = sets.set_id
""", con)
players = pd.read_sql("SELECT * FROM players LEFT JOIN people on players.person_id = people.person_id", con)

In [4]:
model = PlackettLuce()

In [5]:
def run_game(tournament_name, date, tournament_id, tournament_round, pg, tg, game_id):
    # print(f'{tournament_name}, {tournament_round}: {game_id}')
    player_stats = pg[pg['game_id'] == game_id]
    if len(player_stats['team_id'].unique()) != 2:
        return  
    team_stats = tg[tg['game_id'] == game_id]
    if len(team_stats['team_id'].unique()) != 2:
        return  
    team_ids = team_stats['team_id'].unique()
    team1 = []
    team1_scores = []
    team1_stats = player_stats[player_stats['team_id'] == team_ids[0]]
    for i, row in team1_stats.iterrows():
        ratings = pd.read_sql(f'SELECT * from player_ratings where player_id = {row["player_id"]}', qbr)
        if len(ratings) > 0:
            # print(ratings)
            p = model.rating(
                name = row['player_id'], 
                mu = ratings.tail(1).reset_index().at[0, 'mu'], 
                sigma = ratings.tail(1).reset_index().at[0, 'sigma']
                )
        else:
            p = model.rating(name = row['player_id'])
        team1.append(p)
        team1_scores.append(row['pts'])
    team2 = []
    team2_scores = []
    team2_stats = player_stats[player_stats['team_id'] == team_ids[1]]
    for i, row in player_stats[player_stats['team_id'] == team_ids[1]].iterrows():
        ratings = pd.read_sql(f'SELECT * from player_ratings where player_id = {row["player_id"]}', qbr)
        if len(ratings) > 0:
            # print(ratings)
            p = model.rating(
                name = row['player_id'], 
                mu = ratings.tail(1).reset_index().at[0, 'mu'], 
                sigma = ratings.tail(1).reset_index().at[0, 'sigma']
                )
        else:
            p = model.rating(name = row['player_id'])
        team2.append(p)
        team2_scores.append(row['pts'])
    if len(team1) == 0 or len(team2) == 0:
        return
    match = [team1, team2]
    [team1, team2] = model.rate(match, scores = list(team_stats['total_pts']), weights=[team1_scores, team2_scores])
    team1_entries = pd.DataFrame([
        {
            'date': date,
            'tournament_id': tournament_id,
            'round': tournament_round,
            'game_id': game_id,
            'team_id': list(team1_stats['team_id'])[i],
            'player_id': int(list(team1_stats['player_id'])[i]),
            'mu': player_rating.mu,
            'sigma': player_rating.sigma
        }
        for i, player_rating in enumerate(team1)
    ])
    team2_entries = pd.DataFrame([
        {
            'date': date,
            'tournament_id': tournament_id,
            'round': tournament_round,
            'game_id': game_id,
            'team_id': list(team2_stats['team_id'])[i],
            'player_id': int(list(team2_stats['player_id'])[i]),
            'mu': player_rating.mu,
            'sigma': player_rating.sigma
        }
        for i, player_rating in enumerate(team2)
    ])
    team1_entries.to_sql('player_ratings', qbr, if_exists='append', index = False)
    team2_entries.to_sql('player_ratings', qbr, if_exists='append', index = False)
    

In [44]:
cur = qbr.cursor()
cur.execute('DROP TABLE player_ratings')
cur.execute('CREATE TABLE player_ratings (date, tournament_id, round, game_id, team_id, player_id, mu, sigma)')
cur.close()

In [16]:
game_ids = tg[['tournament_name', 'date', 'tournament_id', 'round', 'game_id']].drop_duplicates().sort_values(['date', 'game_id'])
ratings = pd.read_sql('SELECT * from player_ratings', qbr)
prev_games = [int(el) for el in ratings['game_id'].unique()]
game_ids = game_ids[~game_ids.game_id.isin(prev_games)]
game_ids

,tournament_name,date,tournament_id,round,game_id
202276,2011 MOO at Rock Bridge,2011-10-01,29,Round 12,56576
202344,2011 MOO at Rock Bridge,2011-10-01,29,Round 7,56610
202372,2011 MOO at Rock Bridge,2011-10-01,29,Round 9,56624
200952,MAGNI at Brown,2011-10-29,43,Round 6,55768
200954,MAGNI at Brown,2011-10-29,43,Round 6,55769
...,...,...,...,...,...
205528,2024 ACF Winter at Lehigh,2024-11-16,2109,Round 11,114945
205530,2024 ACF Winter at Lehigh,2024-11-16,2109,Round 11,114946
205532,2024 ACF Winter at Lehigh,2024-11-16,2109,Round 11,114947
205534,2024 ACF Winter at Lehigh,2024-11-16,2109,Round 11,114948


In [17]:
ratings = pd.read_sql('SELECT * from player_ratings', qbr)
cur_tournament = ''
for i, row in game_ids.iterrows():
    if cur_tournament != row['tournament_name']:
        print(f"{row['date']}: {row['tournament_name']}")
        cur_tournament = row['tournament_name']
    run_game(row['tournament_name'], row['date'], row['tournament_id'], row['round'], pg, tg, row['game_id'])

2011-10-01: 2011 MOO at Rock Bridge
2011-10-29: MAGNI at Brown
2011-11-12: 2011 Delta Burke at Valencia
2013-02-16: 2013 SCT at MIT, Division II
2013-04-13: 2013 ICT, Division II
2013-05-11: 2013 MUT at Valencia
2014-02-08: 2014 SCT at Chicago, Division II
2014-02-08: 2014 SCT at Chicago, Division I
2014-02-08: 2014 SCT at Chicago, Division II
2014-02-08: 2014 SCT at Chicago, Division I
2014-02-08: 2014 SCT at Chicago, Division II
2014-02-08: 2014 SCT at Chicago, Division I
2014-02-08: 2014 SCT at Chicago, Division II
2014-02-08: 2014 SCT at Chicago, Division I
2014-03-01: 2014 CCCT
2014-03-29: 2014 ICT, Division I
2014-05-09: 2014 MUT at Valencia
2014-11-08: 2014 ACF Fall at Texas
2014-11-22: 2014 Delta Burke at Washington
2015-02-07: 2015 SCT at Carleton University, Division II
2015-02-07: 2015 SCT at Columbia, Division I
2015-02-07: 2015 SCT at Columbia, Division II
2015-02-07: 2015 SCT at Columbia, Division I
2015-02-07: 2015 SCT at Columbia, Division II
2015-02-07: 2015 SCT at Col

In [16]:
len(prev_games)

24752